In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# =========================
# 1) 데이터 로드 + 일별 UNIT 만들기
# =========================
raw = pd.read_csv('C:/taro/1차_프로젝트/통합데이터/SUWON_S_DATA_TABLE.csv')

raw['TA_YMD'] = pd.to_datetime(raw['TA_YMD'], format='%Y%m%d', errors='coerce')
raw = raw.dropna(subset=['TA_YMD'])

daily = (
    raw.groupby('TA_YMD', as_index=False)
       .agg(UNIT=('UNIT', 'sum'))
       .sort_values('TA_YMD')
)

# 날짜 연속성 맞추기(없는 날짜는 NaN -> 보간/대체)
all_days = pd.date_range(daily['TA_YMD'].min(), daily['TA_YMD'].max(), freq='D')
daily = daily.set_index('TA_YMD').reindex(all_days).rename_axis('TA_YMD').reset_index()

# 결측 UNIT 처리: "보간 + 양수 보정"
daily['UNIT'] = daily['UNIT'].interpolate(limit_direction='both')
daily['UNIT'] = daily['UNIT'].clip(lower=0)

# =========================
# 2) 날짜만으로 만들 수 있는 피처 생성 함수
# =========================
def make_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    dt = d['TA_YMD']

    # 날짜 파생
    d['dow'] = dt.dt.dayofweek                 # 0=월 ~ 6=일
    d['month'] = dt.dt.month
    d['day'] = dt.dt.day
    d['weekofyear'] = dt.dt.isocalendar().week.astype(int)
    d['is_weekend'] = (d['dow'] >= 5).astype(int)
    d['is_month_start'] = dt.dt.is_month_start.astype(int)
    d['is_month_end'] = dt.dt.is_month_end.astype(int)

    # 계절성(사인/코사인 인코딩) - 날짜만으로 가능
    doy = dt.dt.dayofyear
    d['doy_sin'] = np.sin(2 * np.pi * doy / 365.25)
    d['doy_cos'] = np.cos(2 * np.pi * doy / 365.25)

    # 과거 UNIT 기반 lag/rolling
    # (예측 시점에 과거 UNIT이 필요 -> 미래는 재귀 예측으로 채움)
    for lag in [1, 7, 14, 28]:
        d[f'lag_{lag}'] = d['UNIT'].shift(lag)

    for w in [7, 14, 28]:
        d[f'roll_mean_{w}'] = d['UNIT'].shift(1).rolling(w).mean()

    # 변화량(트렌드)
    d['diff_1'] = d['UNIT'].diff(1)
    d['diff_7'] = d['UNIT'].diff(7)

    return d

# =========================
# 3) 학습 데이터 생성 (처음 N일은 lag 때문에 제거)
# =========================
feat = make_features(daily)

feature_cols = [
    'dow','month','day','weekofyear','is_weekend','is_month_start','is_month_end',
    'doy_sin','doy_cos',
    'lag_1','lag_7','lag_14','lag_28',
    'roll_mean_7','roll_mean_14','roll_mean_28',
    'diff_1','diff_7'
]

data = feat.dropna(subset=feature_cols + ['UNIT']).copy()

X = data[feature_cols]
y = data['UNIT']

# =========================
# 4) 시계열 분할(랜덤 X)로 평가 + 모델 학습
# =========================
# 마지막 20%를 테스트로
split = int(len(data) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

model = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=6,
    max_iter=600,
    random_state=42
)
model.fit(X_train, y_train)

pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, pred_test)
rmse = mean_squared_error(y_test, pred_test, squared=False)
print(f"[Holdout 평가] MAE={mae:,.2f}  RMSE={rmse:,.2f}")

# =========================
# 5) 날짜만 입력 -> UNIT 예측(미래는 재귀 예측)
# =========================
def predict_unit_by_date(date_str: str) -> float:
    """
    date_str: 'YYYY-MM-DD' 또는 'YYYYMMDD'
    날짜만으로 UNIT 예측.
    과거 날짜: 과거 UNIT 기반 피처로 바로 예측
    미래 날짜: 하루씩 예측값을 UNIT에 채워 넣으며 목표일까지 재귀 예측
    """
    s = date_str.strip()
    if len(s) == 8 and s.isdigit():
        target_date = pd.to_datetime(s, format='%Y%m%d')
    else:
        target_date = pd.to_datetime(s)

    # daily 범위를 벗어나면(미래) 시뮬레이션용으로 날짜 확장
    base = daily.copy()
    last_date = base['TA_YMD'].max()

    if target_date > last_date:
        future_days = pd.date_range(last_date + pd.Timedelta(days=1), target_date, freq='D')
        add = pd.DataFrame({'TA_YMD': future_days, 'UNIT': np.nan})
        base = pd.concat([base, add], ignore_index=True)

    # 목표일까지 순차적으로 feature 만들고, 미래 구간은 예측해서 UNIT 채움
    base = base.sort_values('TA_YMD').reset_index(drop=True)

    for i in range(len(base)):
        dt = base.loc[i, 'TA_YMD']
        if dt > last_date and pd.isna(base.loc[i, 'UNIT']):
            tmp = make_features(base[['TA_YMD','UNIT']].iloc[:i+1].copy())
            row = tmp.iloc[i:i+1]

            if row[feature_cols].isna().any(axis=1).iloc[0]:
                # lag가 부족하면 예측 불가 (데이터 초반/너무 짧은 데이터)
                raise ValueError("예측에 필요한 과거 데이터(최소 28일)가 부족합니다.")

            x = row[feature_cols]
            yhat = float(model.predict(x)[0])
            base.loc[i, 'UNIT'] = max(0.0, yhat)  # 음수 방지

        if dt == target_date:
            # target_date의 UNIT이 실제값(과거)이든 예측값(미래)이든
            # 그 날짜 feature로 '예측 UNIT' 출력(과거도 예측치 반환)
            tmp = make_features(base[['TA_YMD','UNIT']].iloc[:i+1].copy())
            row = tmp.iloc[i:i+1]
            if row[feature_cols].isna().any(axis=1).iloc[0]:
                raise ValueError("해당 날짜 예측에 필요한 과거 데이터(최소 28일)가 부족합니다.")
            return float(model.predict(row[feature_cols])[0])

    raise ValueError("입력 날짜를 처리하지 못했습니다. 날짜 형식을 확인하세요.")

# =========================
# 6) 실행 예시
# =========================
if __name__ == "__main__":
    d = input("예측할 날짜 입력 (YYYY-MM-DD 또는 YYYYMMDD): ")
    yhat = predict_unit_by_date(d)
    print(f"[예상 UNIT] {yhat:,.0f}")

[Holdout 평가] MAE=98,232.87  RMSE=225,713.85


ValueError: 예측에 필요한 과거 데이터(최소 28일)가 부족합니다.

In [5]:
if __name__ == "__main__":
    d = input("예측할 날짜 입력 (YYYY-MM-DD 또는 YYYYMMDD): ")
    yhat = predict_unit_by_date(d)
    print(f"[예상 UNIT] {yhat:,.0f}")

[예상 UNIT] 5,449,180
